In [1]:
# TO DO:
# 
# Extend single link info builder:
#        email, name, affiliation, department, paper title (PMID), specialty???
#
# Extend search page navigator:
#        
#        optimize run time
#        create more search items and refined search options like from website??
#
#
#


import requests
import pandas as pd
from bs4 import BeautifulSoup

def singleLinkInfoListBuilder(URLextender):

    URL = "https://pubmed.ncbi.nlm.nih.gov/" + str(URLextender) + "/"
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    tempInfoList = []
    finalInfoList = []
    
    
    paperTitle = paperTitleFinder(URL)
    paperID = IDGetter(URL)
    nameList = nameGetter(URL)

    results = soup.find(id = "full-view-expanded-authors")
    
    if len(soup.find_all(id = "full-view-expanded-authors")) == 0:
        finalInfoList.append("")
        return finalInfoList
    
    if (len(results.find_all("ul")) == 0):
        finalInfoList.append("")
        return finalInfoList
    
    results = results.ul
    results = results.find_all("li")

    tempList = [None] * len(results)
    

    for i in range(len(results)):
        tempList[i] = str(results[i])
        affiliationNumber = tempList[i][37]
        for j in nameList:
            if j[1] == affiliationNumber:
                person = j[0]
        tempList[i] = tempList[i][65:-6]
        tempList[i] = tempList[i].split(",")
        if "@" in tempList[i][-1]:
            
            email = finalEmailCleaner(tempList[i][-1])
            
            try:
                tempInfoList.append([person, tempList[i][0] + ", " + tempList[i][1], email])  
            except IndexError:
                
                print(paperTitle + "FAILED")
                print(tempList[i])
    
    if len(tempInfoList) == 0:
        finalInfoList.append("")
    
    else:
        finalInfoList.append(paperTitle)
        finalInfoList.append(paperID)
        finalInfoAppender = []
        for i in range(len(tempInfoList)):
            if tempInfoList[i] not in finalInfoAppender:
                finalInfoAppender.append(tempInfoList[i])
        finalInfoList.append(finalInfoAppender)    
    
    return finalInfoList


def searchPageURLFinder(searchPageURL):
    
    resultsList = []
    for i in range(1, 5):
        
        searchPageURL += "&page=" + str(i)
        
        page = requests.get(searchPageURL)
        soup = BeautifulSoup(page.content, 'html.parser')
    
        results = soup.find_all(string = isADataChunkID)
    
        for i in results:
            resultsList.append(i)
    
    infoList = []
    
    for i in resultsList:
        infoList.append(singleLinkInfoListBuilder(i))
        
    return infoList
    
def isADataChunkID(s):
    
    while True:
        try:
            tempExt = int(s)
            return (len(str(tempExt)) == 8 or len(str(tempExt)) == 7 or len(str(tempExt)) == 6)
        except ValueError:
            return False
        
def getUserInput():
    
    search = str(input("Please enter your desired search term: "))
    listified = search.split(" ")
    result = ""
    for i in range(len(listified)):
        if i < len(listified) - 1:
            result += listified[i] + "+"
        else:
            result += listified[i]
    
    return "https://pubmed.ncbi.nlm.nih.gov/?term=" + result + "&size=200"
                                 
def finalEmailCleaner(item):
                                 
    itemList = item.split(" ")
    return itemList[-1]
                                 
def paperTitleFinder(URL):
    
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find(class_ = "heading-title")
    results = str(results)
    results = results.split("\n")
    return results[7][2:]
              
    
def nameGetter(URL):
    
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')

    results = soup.find_all(class_ = "authors-list-item")
    affiliationNumbers = soup.find_all(class_ = "affiliation-link")
    affiliationNumbers = affiliationNumbers[:(len(affiliationNumbers) // 2)]
    results = results[:len(results) // 2]

    nameList = []

    for i in range(len(results)):
    
        names = results[i].a
        names = str(names)
        
        try:
            affiliationNumber = str(affiliationNumbers[i])[47]
            name = names[90:]
            name = name.split('"')
        
            nameList.append([])
            nameList[i].append(name[1])
            nameList[i].append(affiliationNumber)
        except IndexError:
            break
    
    return nameList

def IDGetter(URL):
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')

    results = soup.find(class_ = "current-id")

    results = str(results)

    IDFinder = False
    IDnum = ""

    for char in results:
    
        if char == "<":
            IDFinder = False
        if IDFinder:
            IDnum += char
        if char == ">":
            IDFinder = True
        
    return IDnum

def finalInfoListCleaner(finalList):
    
    stillCleaning = True
    while stillCleaning:
        try:
            finalList.remove([''])
        except ValueError:
            stillCleaning = False
            
    return finalList

In [2]:
searchTopic = getUserInput()

emailList = searchPageURLFinder(searchTopic)

emailList = finalInfoListCleaner(emailList)

print(emailList)

Please enter your desired search term: lung cancer
[Screening for early detection of lung cancer: conflict between clinical and public health viewpoints]FAILED
['Országos Tisztifoorvosi Hivatal Budapest. dobrossy@freemail.h']
[Treatment of non small cell lung cancer]FAILED
['Klinika nemocí plicních a tuberkulózy LF MU a FN Brno. jskric@fnbrno.c']


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [5]:


print(len(emailList))

49


<strong class="current-id" title="PubMed ID">18765141</strong>
18765141


In [5]:
37

37